Analisa os dados de pedidos e entregas focando no desempenho operacional e cumprimento de SLA, buscando identificar padrões de atraso e possíveis gargalos e definindo:
- Quantidade de pedidos
- Taxa média de atrasos
- Atrasos de cada transportadora
- Atrasos de cada região

In [4]:
# Lê e carrega arquivos
# ---------------------------------------------------------
import pandas as pd
# ---------------------------------------------------------
customers = pd.read_csv("../data/raw/customers.csv")
carriers = pd.read_csv("../data/raw/carriers.csv")
orders = pd.read_csv("../data/raw/orders.csv")
deliveries = pd.read_csv("../data/raw/deliveries.csv")

In [5]:
# Realiza validações e análises dos dados:
# ---------------------------------------------------------
# Validações:
# Informações sobre os dados:
customers.info()
carriers.info()
orders.info()
deliveries.info()

# Dados nulos:
print("\nValores nulos em customers:")
print(customers.isna().sum())
print("\nValores nulos em carriers:")
print(carriers.isna().sum())
print("\nValores nulos em orders:")
print(orders.isna().sum())
print("\nValores nulos em deliveries:")
print(deliveries.isna().sum())

# ---------------------------------------------------------
# Quantia de pedidos:
num_orders = orders.shape[0]
print("Quantia de pedidos Total: ", num_orders)

# Quantia de pedidos atrasados:
num_delivered_late = sum(deliveries["delivered_late"]==True)
print("Pedidos atrasados Totais: ", num_delivered_late, end=" ")
print("%.2f%% dos pedidos totais" %(num_delivered_late*100/num_orders))

# Construção da base analítica unificada
# ---------------------------------------------------------
orders_full = (
    orders
    .merge(customers, on="customer_id", how="left")
    .merge(deliveries, on="order_id", how="left")
    .merge(carriers, on="carrier_id", how="left")
)
print("\nValores nulos após os merges:")
print(orders_full.isna().sum())


# Atrasos de cada transportadora
carrier_late_summary = (orders_full.groupby("carrier_name").agg(total_orders=("order_id", "count"),late_orders=("delivered_late", "sum")))
carrier_late_summary["late_rate_pct"] = (carrier_late_summary["late_orders"]/ carrier_late_summary["total_orders"] * 100)
carrier_late_summary["pct_of_total_late"] = (carrier_late_summary["late_orders"]/ num_delivered_late * 100)
carrier_late_summary = carrier_late_summary.sort_values("late_rate_pct", ascending=False)
carrier_late_summary

# Atrasos de cada região: 
region_late_summary = (orders_full.groupby("region").agg(total_orders=("order_id", "count"),late_orders=("delivered_late", "sum")))
region_late_summary["late_rate_pct"] = (region_late_summary["late_orders"]/ region_late_summary["total_orders"] * 100)
region_late_summary["pct_of_total_late"] = (region_late_summary["late_orders"]/ num_delivered_late * 100)
region_late_summary = region_late_summary.sort_values("late_rate_pct", ascending=False)
region_late_summary

# Atrasos por SLA
sla_late_summary = (orders_full.groupby("sla_days").agg(total_orders=("order_id", "count"),late_orders=("delivered_late", "sum")))
sla_late_summary["late_rate_pct"] = (sla_late_summary["late_orders"]/ sla_late_summary["total_orders"] * 100)
sla_late_summary["pct_of_total_late"] = (sla_late_summary["late_orders"]/ num_delivered_late * 100)
sla_late_summary = sla_late_summary.sort_values("late_rate_pct", ascending=False)
sla_late_summary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  1000 non-null   int64 
 1   region       1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carrier_id    5 non-null      int64  
 1   carrier_name  5 non-null      object 
 2   delay_risk    5 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 252.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       5000 non-null   int64 
 1   customer_id    5000 non-null   int64 
 2   order_date     

,total_orders,late_orders,late_rate_pct,pct_of_total_late
sla_days,,,,
7,983,182,18.514751,20.177384
3,2001,360,17.991004,39.911308
5,2016,360,17.857143,39.911308


In [6]:
# Análise de atrasos por SLA e transportadora
# ---------------------------------------------------------

sla_carrier_summary = (
    orders_full
    .groupby(["carrier_name", "sla_days"])
    .agg(total_orders=("order_id", "count"),late_orders=("delivered_late", "sum"))
    .reset_index()
)
sla_carrier_summary["late_rate_pct"] = (sla_carrier_summary["late_orders"]/ sla_carrier_summary["total_orders"] * 100)
sla_carrier_summary = sla_carrier_summary.sort_values(["sla_days", "late_rate_pct"],ascending=[True, False])
sla_carrier_summary

,carrier_name,sla_days,total_orders,late_orders,late_rate_pct
9,SulCargo,3,411,118,28.710462
6,RápidoX,3,400,97,24.250000
3,EntregaJá,3,404,64,15.841584
0,BrasilExpress,3,362,41,11.325967
12,TransLog,3,424,40,9.433962
7,RápidoX,5,416,104,25.000000
10,SulCargo,5,381,94,24.671916
4,EntregaJá,5,387,69,17.829457
13,TransLog,5,426,48,11.267606
1,BrasilExpress,5,406,45,11.083744


In [7]:
# Simulação: remoção da pior transportadora por SLA
# ---------------------------------------------------------

# Base geral
baseline = (
    orders_full
    .groupby("sla_days")
    .agg(
        total_orders=("order_id", "count"),
        late_orders=("delivered_late", "sum")
    )
)
baseline["late_rate_pct"] = (baseline["late_orders"] / baseline["total_orders"] * 100)
baseline = baseline.reset_index()


# Identifica a pior transportadora por SLA
worst_carrier_by_sla = (
    orders_full
    .groupby(["sla_days", "carrier_name"])
    .agg(total_orders=("order_id", "count"),late_orders=("delivered_late", "sum"))
    .reset_index()
)

worst_carrier_by_sla["late_rate_pct"] =(worst_carrier_by_sla["late_orders"]  / worst_carrier_by_sla["total_orders"] * 100)

idx = worst_carrier_by_sla.groupby("sla_days")["late_rate_pct"].idxmax()
worst_carriers = worst_carrier_by_sla.loc[idx]

print("Pior transportadora por SLA:")
print(worst_carriers[["sla_days", "carrier_name", "late_rate_pct"]])




# Simulação: remover a pior transportadora de cada SLA
orders_simulated = orders_full.merge(worst_carriers[["sla_days", "carrier_name"]],on=["sla_days", "carrier_name"],how="left",indicator=True)
orders_simulated = orders_simulated[orders_simulated["_merge"] == "left_only"]


# Métricas após simulação
simulated = (
    orders_simulated
    .groupby("sla_days")
    .agg(total_orders=("order_id", "count"),late_orders=("delivered_late", "sum"))

    .reset_index()
)

simulated["late_rate_pct"] = (simulated["late_orders"] / simulated["total_orders"] * 100)


# Comparação
comparison = baseline.merge(simulated,on="sla_days",suffixes=("_before", "_after"))
comparison["improvement_pct_points"] = (comparison["late_rate_pct_before"]- comparison["late_rate_pct_after"])
# ---------------------------------------------------------
comparison

Pior transportadora por SLA:
    sla_days carrier_name  late_rate_pct
3          3     SulCargo      28.710462
7          5      RápidoX      25.000000
13         7     SulCargo      31.250000


,sla_days,total_orders_before,late_orders_before,late_rate_pct_before,total_orders_after,late_orders_after,late_rate_pct_after,improvement_pct_points
0,3,2001,360,17.991004,1590,242,15.220126,2.770879
1,5,2016,360,17.857143,1600,256,16.000000,1.857143
2,7,983,182,18.514751,791,122,15.423515,3.091236


In [10]:
# Exportação dos dados para Power BI
# ---------------------------------------------------------
import os
# ---------------------------------------------------------
OUTPUT_PATH = "data/processed"
os.makedirs(OUTPUT_PATH, exist_ok=True)


# KPIs gerais
# ---------------------------------------------------------

kpis = {
    "total_orders": num_orders,
    "late_orders": num_delivered_late,
    "late_rate_pct": round(num_delivered_late * 100 / num_orders, 2)
}

kpis_df = pd.DataFrame([kpis])
kpis_df.to_csv(f"{OUTPUT_PATH}/kpis_gerais.csv", index=False)


# Atrasos por região
# ---------------------------------------------------------

region_summary = (
    orders_full
    .groupby("region")
    .agg(
        total_orders=("order_id", "count"),
        late_orders=("delivered_late", "sum")
    )
    .reset_index()
)

region_summary["late_rate_pct"] = (
    region_summary["late_orders"]
    / region_summary["total_orders"] * 100
)

region_summary.to_csv(f"{OUTPUT_PATH}/atrasos_por_regiao.csv",index=False)


# Atrasos por transportadora
# ---------------------------------------------------------

carrier_summary = (
    orders_full
    .groupby("carrier_name")
    .agg(
        total_orders=("order_id", "count"),
        late_orders=("delivered_late", "sum")
    )
    .reset_index()
)

carrier_summary["late_rate_pct"] = (
    carrier_summary["late_orders"]
    / carrier_summary["total_orders"] * 100
)

carrier_summary.to_csv(f"{OUTPUT_PATH}/atrasos_por_transportadora.csv",index=False)

# Atrasos por SLA
# ---------------------------------------------------------

sla_summary = (
    orders_full
    .groupby("sla_days")
    .agg(
        total_orders=("order_id", "count"),
        late_orders=("delivered_late", "sum")
    )
    .reset_index()
)

sla_summary["late_rate_pct"] = (
    sla_summary["late_orders"]
    / sla_summary["total_orders"] * 100
)

sla_summary.to_csv(f"{OUTPUT_PATH}/atrasos_por_sla.csv",index=False)


# SLA x Transportadora
# ---------------------------------------------------------

sla_carrier_summary.to_csv(f"{OUTPUT_PATH}/sla_x_transportadora.csv", index=False)


# Simulação de decisão
# ---------------------------------------------------------
comparison.to_csv(f"{OUTPUT_PATH}/simulacao_remocao_pior_transportadora.csv",index=False)

# Tabela fato:
# ---------------------------------------------------------
orders_full[["order_id", "region", "carrier_name", "sla_days", "delivered_late"]].to_csv("data/processed/fato_entregas.csv", index=False)

# ---------------------------------------------------------
print("Arquivos exportados com sucesso para data/processed/")

Arquivos exportados com sucesso para data/processed/
